In [1]:
#SVM 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy import genfromtxt
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA, KernelPCA
from skimage.feature import hog

# Our dataset and targets
my_data = genfromtxt('data/processed_nist_data.csv', delimiter=',')
#my_data.shape
X = my_data[:, 1:901]
Y = my_data[:, 0]

# Feature selection: remove features with low variance (Boolean variables --> Bernoulli variables)
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = sel.fit_transform(X)

In [3]:
# Feature extraction 

# PCA
#pca = PCA(n_components=X.shape[1])
#pca.fit(X)

# Kernel PCA
kpca = KernelPCA(n_components=(X.shape[1])
kpca.fit(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=0)

SyntaxError: invalid syntax (<ipython-input-3-b5c93eb18f74>, line 9)

In [ ]:
# fit the model and testing with 3 different types of kernels
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel, gamma='auto')
    clf.fit(X_train, y_train)
    # Test
    y_pred = clf.predict(X_test)
    # Accuracy
    print(accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)*100)
    
# Results 
# raw pixels: linear 95.4%, polynomial 79.65%, Radial Basis Function (RBF) 95.85% 
# feature selection: linear 93.7%, polynomial 88.15%, Radial Basis Function (RBF) 96.05% 

In [ ]:
X_train.shape